In [1]:
import sympy as sp
import numpy as np
import pandas as pd
from collections import Counter
import math


    

In [14]:
def FM(A):
    n = len(A)
    lam,vlam = (np.linalg.eig(A))
    for i in range(len(lam)):
        lam[i] = round(lam[i],5)
    k_lam = Counter(lam) # собственные числа и их кратности
    print(lam,k_lam)
    list_b = list(sp.var('b0 b1 b2 b3 b4 b5 b6 b7 b8 b9 b10',real = True))
    sp.var('t',real = True) # полиномы, переменные
    sp.var('lamb')
     
    # правая части уравнений для поиска полиномов
    right = list_b[0]
    for i in range(1, n):
        right += list_b[i]*lamb**i
    right -= sp.exp(t*lamb)
    # генерируем систему
    B = []
    i = 0
    while i < n:
        for x in k_lam:
            if sp.im(x) >= 0:
                for r in range(k_lam[x]):
                    d_right = right
                    for k in range(r):
                        d_right = sp.diff(d_right,lamb)
                        
                    d_right = d_right.subs(lamb,x)
                    if sp.im(x) == 0:
                        B.append(sp.simplify(d_right))
                    else:
                        B.append(sp.simplify(sp.re(d_right)))
                        B.append(sp.simplify(sp.im(d_right)))
            i += k_lam[x]
            
            
    ans, = sp.simplify(sp.linsolve(B,list_b))
    An = []
    for i in range(n):
        Ak = np.eye(n)
        for j in range(i):
            Ak = np.matmul(Ak,A)
        An.append(Ak)
    Y = np.zeros((n,n))
    for i in range(n):
        Y = Y + ans[i]*sp.Matrix(An[i])
    Y = sp.simplify(Y)
    return (Y)

In [16]:
A = [[0,1,0],[0,0,1],[0,0,0]]
FM(A)

[0. 0. 0.] Counter({0.0: 3})


Matrix([
[1.0, 1.0*t, 0.5*t**2],
[  0,   1.0,    1.0*t],
[  0,     0,      1.0]])

Построение программного управления

In [5]:
def PrU(P,Q,Y,f,x0,x1,Time):
    Yinv = np.linalg.inv(Y)
    B = Yinv*Q
    A = sp.integrate(B*B.T,t)
    A = ()
    nl = Yinv*x1 - x0 - sp.integrate(Yinv*f,t)